In [1]:
#La Agencia Nacional de Estadísticas de Buenos Aires recolecta información de nacimientos
#cuando los padres registran a sus hijos en el registro civil a partir de una encuesta. Esa
#información se encuentra disponible para su análisis en un csv con el siguiente formato
#(dia_nacimiento, mes_nacimiento, anio_nacimiento, peso_al_nacer, longitud_al_nacer,
#id_hospital, tipo_parto), donde el tipo de parto 1 es natural y 2 es cesárea.
#Por otro lado la agencia cuenta con información histórica de los hospitales en otro csv con
#siguiente formato (id_hospital, dirección, promedio_nacimientos_mensual).

In [2]:
import pandas as pd

In [3]:
nacimientos = pd.read_csv('nacimientos.csv')

In [4]:
hospitales = pd.read_csv('hospitales.csv')

In [5]:
#Calcular la cantidad de nacimientos para cada uno de los hospitales para el mes de
#Octubre de 2017 e indicar aquellos hospitales que superan el promedio de nacimientos
#mensuales

In [6]:
nacimientos.head()

,dia_nacimiento,mes_nacimiento,anio_nacimiento,peso_al_nacer,longitud_al_nacer,id_hospital,tipo_parto
0,23,10,2018,234,56,1,1
1,1,10,2016,24421,56,2,2
2,4,10,2016,2424,45,3,1
3,5,10,2016,2434,45,4,2
4,14,10,2016,1234,65,1,1


In [7]:
hospitales.head()

,id_hospital,direccion,promedio_nacimientos_mensual
0,1,Paraguay 1238,8
1,2,Chacras 123,2
2,3,Corrientes 1200,5
3,4,Belgrano 122,3


In [8]:
nacimientos_octubre_2017 = nacimientos[(nacimientos['anio_nacimiento'] == 2017) & (nacimientos['mes_nacimiento'] == 10)].copy()

In [9]:
nacimientos_octubre_2017

,dia_nacimiento,mes_nacimiento,anio_nacimiento,peso_al_nacer,longitud_al_nacer,id_hospital,tipo_parto
6,6,10,2017,4234,65,3,1
7,1,10,2017,4324,65,4,2
9,4,10,2017,423,34,2,2
10,4,10,2017,423,67,3,1
11,4,10,2017,234,77,4,2
12,23,10,2017,423,67,1,1
13,23,10,2017,423,67,2,2


In [10]:
nacimientos_octubre_2017.loc[:,'cantidad_nacimientos'] = 1

In [11]:
nacimientos_por_hospital = nacimientos_octubre_2017.groupby('id_hospital').agg({'cantidad_nacimientos':'count'})
nacimientos_por_hospital

,cantidad_nacimientos
id_hospital,
1,1
2,2
3,2
4,2


In [12]:
nacimientos_vs_hospitales = nacimientos_por_hospital.merge(hospitales, on='id_hospital', how='inner')
nacimientos_vs_hospitales

,id_hospital,cantidad_nacimientos,direccion,promedio_nacimientos_mensual
0,1,1,Paraguay 1238,8
1,2,2,Chacras 123,2
2,3,2,Corrientes 1200,5
3,4,2,Belgrano 122,3


In [13]:
nacimientos_vs_hospitales.drop(columns='direccion', inplace=True)

In [14]:
print(nacimientos_vs_hospitales[nacimientos_vs_hospitales['cantidad_nacimientos'] >= nacimientos_vs_hospitales['promedio_nacimientos_mensual']])

   id_hospital  cantidad_nacimientos  promedio_nacimientos_mensual
1            2                     2                             2


In [15]:
#Comparando el mes de Octubre de 2017 indicar programáticamente si se incremento el porcentaje
#de cesáreas con respecto a ese mes del año 2016

In [16]:
#busco nacimientos en octubre 2016
nacimientos_octubre_2016 = nacimientos[(nacimientos['anio_nacimiento'] == 2016) & (nacimientos['mes_nacimiento'] == 10)].copy()

In [17]:
nacimientos_octubre_2016['cantidad_nacimientos']=1
nacimientos_octubre_2016.drop(columns=['dia_nacimiento','mes_nacimiento','peso_al_nacer','id_hospital', 'longitud_al_nacer'],inplace=True)

In [18]:
nacimientos_octubre_2016 = nacimientos_octubre_2016.groupby(['anio_nacimiento','tipo_parto']).agg({'cantidad_nacimientos':'sum'}).reset_index()

In [19]:
cantidad_total_nacimientos_2016 = nacimientos_octubre_2016['cantidad_nacimientos'].sum()
cantidad_total_nacimientos_2016

5

In [20]:
nacimientos_octubre_2016

,anio_nacimiento,tipo_parto,cantidad_nacimientos
0,2016,1,2
1,2016,2,3


In [21]:
nacimientos_octubre_2016['porcentaje_tipo_parto'] = nacimientos_octubre_2016['cantidad_nacimientos']/cantidad_total_nacimientos_2016
nacimientos_octubre_2016

,anio_nacimiento,tipo_parto,cantidad_nacimientos,porcentaje_tipo_parto
0,2016,1,2,0.4
1,2016,2,3,0.6


In [22]:
nacimientos_octubre_2017.drop(columns=['dia_nacimiento','mes_nacimiento','peso_al_nacer','id_hospital', 'longitud_al_nacer'],inplace=True)
nacimientos_octubre_2017

,anio_nacimiento,tipo_parto,cantidad_nacimientos
6,2017,1,1
7,2017,2,1
9,2017,2,1
10,2017,1,1
11,2017,2,1
12,2017,1,1
13,2017,2,1


In [23]:
nacimientos_octubre_2017 = nacimientos_octubre_2017.groupby(['anio_nacimiento','tipo_parto']).agg({'cantidad_nacimientos':'sum'}).reset_index()
nacimientos_octubre_2017

,anio_nacimiento,tipo_parto,cantidad_nacimientos
0,2017,1,3
1,2017,2,4


In [24]:
nacimientos_octubre_2017['porcentaje_tipo_parto'] = nacimientos_octubre_2017['cantidad_nacimientos']/nacimientos_octubre_2017['cantidad_nacimientos'].sum()
nacimientos_octubre_2017

,anio_nacimiento,tipo_parto,cantidad_nacimientos,porcentaje_tipo_parto
0,2017,1,3,0.428571
1,2017,2,4,0.571429


In [25]:
aumentaron_cesareas=nacimientos_octubre_2017[nacimientos_octubre_2017['tipo_parto']==2]['porcentaje_tipo_parto'] > nacimientos_octubre_2016[nacimientos_octubre_2016['tipo_parto']==2]['porcentaje_tipo_parto']
aumentaron_cesareas

1    False
Name: porcentaje_tipo_parto, dtype: bool